In [1]:
%pip install --upgrade pymilvus[model] langchain-milvus langchain-openai
%pip install -qU langchain_community pypdf

zsh:1: no matches found: pymilvus[model]
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.0.63 requires langchain-community<0.3.0,>=0.2.6, but you have langchain-community 0.3.17 which is incompatible.
langchain-experimental 0.0.63 requires langchain-core<0.3.0,>=0.2.10, but you have langchain-core 0.3.35 which is incompatible.
langchain-anthropic 0.3.1 requires anthropic<1,>=0.41.0, but you have anthropic 0.37.1 which is incompatible.
langchain-openai 0.1.20 requires langchain-core<0.3.0,>=0.2.26, but you have langchain-core 0.3.35 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_ollama import OllamaLLM

In [3]:
CONNECTION_URI = "http://localhost:19530"

In [4]:
ollama_embedding_func = OllamaLLM(model="moondream")

In [ ]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Ollama Setup
def setup_ollama_pdf_vectorstore(file_path, embedding_model="moondream"):
    try:
        # 1. Load PDF
        loader = PyMuPDFLoader(file_path)
        docs = loader.load()

        # 2. Text Splitting (Optional but recommended)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        splits = text_splitter.split_documents(docs)

        # 3. Ollama Embeddings
        embeddings = OllamaEmbeddings(
            model=embedding_model,  # You can change this to other Ollama models
            num_gpu=16  # Optional: specify GPU usage
        )

        # 4. Create Chroma Vector Store
        vector_store = Chroma.from_documents(
            documents=splits,
            embedding=embeddings,
            collection_name="ollama_pdf_collection"
        )

        return vector_store

    except Exception as e:
        print(f"Error processing PDF: {e}")
        return None

files = os.listdir("papers")
print(files)

vector_store = setup_ollama_pdf_vectorstore(f"papers/{files[0]}")
print(f"Vector store for {files[0]} created successfully")
print("")

['2303.05510v1.pdf']


In [ ]:
# Perform Similarity Search
contexts = ""
query = input("Enter your query: ")

if vector_store:
    results = vector_store.similarity_search(query, k=10)
    
    for doc in results:
        print("\n--- Relevant Document Excerpt ---")
        print(doc.page_content)
        contexts = contexts + doc.page_content


--- Relevant Document Excerpt ---
a tree search algorithm inspired by Monte-Carlo tree search. We evaluate our algorithm and em-
pirically show that our algorithm generates programs of higher quality compared with competing
baseline algorithms in different settings. We also design model structure-speciﬁc caching mecha-
nisms which contribute to saving computational expenses. We show that our algorithm is versatile
and can generate codes under objectives other than the pass rate without ﬁnetuning the Transformer.
We hope our work can inspire more ways of incorporating planning algorithms into the Transformer
generation process for code generation or other problems.
One limitation of our algorithm is its reliance on test cases, although we ﬁnd that even a small
number of test cases can help ﬁnd better solutions (Sec. A, Table 7). To address this limitation, we
provided results to show that our algorithm can take advantage of automatically-generated test cases

--- Relevant Document Exce

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize Ollama Chat Model
chat_model = ChatOllama(
    model="llama3.2:1b",  # You can change to other models like mistral, phi3
    temperature=0.7,  # Creativity level
)

# Simple Chat Interaction
messages = [
    SystemMessage(content="You are a helpful assistant for my research."),
    HumanMessage(content=f"Query + {query} with the context + {contexts}")
]

# Generate Response
response = chat_model.invoke(messages)
print("--- Ollama Response ---")
print(response.content)

AttributeError: 'ModelPrivateAttr' object has no attribute 'chat'